In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup as BS

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_20 = pd.read_excel('../Data/2020_precinct_detail.xlsx')

In [ ]:
df_20.head(3)

In [ ]:
senate_20 = df_20.loc[df_20['OFFICENAME'] == ('United States Senate')]

In [ ]:
senate_20

In [ ]:
df_2002 = pd.read_stata('../Data/2002_precinct_detail.dta')

In [ ]:
df_2002

In [ ]:
print('Dem total: ' + str(df_2002['g2002_GOV_dv'].sum()))
print('Rep total: ' + str(df_2002['g2002_GOV_rv'].sum()))

In [ ]:
d = {'g2002_GOV_dv': ['sum'], 'g2002_GOV_rv': ['sum']}
res = df_2002.groupby('county').agg(d)

In [ ]:
res

In [ ]:
all_house = pd.read_csv('../Data/1976-2020-house.csv')

In [ ]:
all_house.head(3)

In [ ]:
tn_house7620 = all_house.loc[all_house['state_po'] == ('TN')]
tn_house7620

In [ ]:
tn_house7620.groupby(['year', 'party'])['candidatevotes'].agg([('candidatevotes', 'sum')]).reset_index()


In [ ]:
stateleg_df = pd.read_stata('../Data/1967-2016_stateleg.dta')

In [ ]:
stateleg_df.loc[stateleg_df['state'] == 'Tennessee']

In [ ]:
(stateleg_df.loc[stateleg_df['state'] == 'Tennessee'])['year'].value_counts()

In [ ]:
gov_df = pd.read_excel('../Data/1961-2010_gov.xlsx')

In [ ]:
gov_df

In [ ]:
tn_gov_df = gov_df.loc[gov_df['state'] == 'Tennessee']

In [ ]:
tn_gov_df.loc[tn_gov_df['year'] == 2010]

In [ ]:
pbal_df = pd.read_excel('../Data/state_part_bal_1937-2011.xlsx')

In [ ]:
pbal_df

In [ ]:
tn_pbal_df = pbal_df.loc[pbal_df['state'] == 'Tennessee']

In [ ]:
tn_pbal_df

In [ ]:
URL = 'https://uselectionatlas.org/RESULTS/compare.php?year=1982&fips=47&f=0&off=5&elect=0&type=state'

response = requests.get(URL)

In [ ]:
type(response)

In [ ]:
response.status_code

In [ ]:
response.text

In [ ]:
soup = BS(response.text)

In [ ]:
soup

In [ ]:
print(soup.prettify())

In [ ]:
soup.findAll('table')

In [ ]:
soup.find('table', attrs={'class' : 'datatable'})

In [ ]:
table_html = str(soup.find('table', attrs={'id' : 'datatable'}))

from IPython.core.display import HTML

HTML(table_html)



In [ ]:
gov_df = pd.read_html(str(soup.find('table', attrs={'id' : 'datatable'})))[0]

In [ ]:
gov_df

In [ ]:
gov_df.drop(columns = ['Map', 'Pie', 'I'], inplace=True)

In [ ]:
gov_df.drop(gov_df.index[[0, 11, 12, 13, 14, 15]], inplace=True)

In [ ]:
gov_df

In [ ]:
gov_df[gov_df.columns[0:5]] = gov_df[gov_df.columns[0:5]].astype('int64')


In [ ]:
gov_df

In [ ]:
gov_df[gov_df.columns[10:15]] = gov_df[gov_df.columns[10:15]].astype('int64')

In [ ]:
gov_df

In [ ]:
gov_df['office'] = 'Gov'

In [ ]:
gov_df.rename({'Year': 'year', 'Total': 'total_votes', 'Dem': 'Dem%', 'Rep': 'Rep%', 'Oth.': 'Other%', 'Dem.1': 'Dem', 'Rep.1': 'Rep', 'Other': 'Oth_total'}, axis=1, inplace=True) 

In [ ]:
gov_df.head(3)

In [ ]:
gov_df['Other'] = gov_df['Ind.1'] + gov_df['Oth_total']

In [ ]:
gov_df.head(3)

In [ ]:
gov_df = gov_df[['year', 'office', 'total_votes', 'Rep', 'Dem', 'Other', 'Rep%', 'Dem%', 'Other%']]

In [ ]:
gov_df.head(3)

In [ ]:
gov_df['Rep%'] = (gov_df['Rep%'].str.rstrip('%').astype('float')).round(1)
gov_df['Dem%'] = (gov_df['Dem%'].str.rstrip('%').astype('float')).round(1)
gov_df['Other%'] = (gov_df['Other%'].str.rstrip('%').astype('float')).round(1)

In [ ]:
gov_df

In [ ]:
gov_df.info()

In [ ]:
# gov_df.to_csv("gov_statewide_clean.csv", index=False)

add 'win' column to denote winning party

for index, row in gov_df.iterrows():
    if row.D == 1:
        gov_df.loc[index, 'win'] = 'D'
    elif row.D == 2:
        gov_df.loc[index, 'win'] = 'R'
    else:
        gov_df.loc[index, 'win'] = 'O'

In [ ]:
gov_df